In [1]:
import glob
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
with open('./species_html/abarema_brachystachya_(dc)_barneby_&_jwgrimes.html','r') as f:
    html = f.read()

In [3]:
bs = BeautifulSoup(html)

# Finding fieldsets

In [4]:
fieldsets = [ legend.findParent() for legend in bs.findAll('legend') ]

## Fieldset 1 (Identificação)

In [5]:
labels = fieldsets[0].findAll('td',{ 'class': "pg_infogeral_label"})
values = fieldsets[0].findAll('td',{ 'class': "pg_infogeral_valor"})

fs1 = list(zip( [ l.text for l in labels], [v.text for v in values] ))
fs1

[('Espécie: ', 'Abarema brachystachya(DC.) Barneby & J.W.Grimes'),
 ('Nome Popular: ', 'Timbuva, Olho-cabra-azul'),
 ('Sinonímia: ',
  'Inga brachystachya DC., Abarema obovata (Benth.) Barneby & J.W.Grimes, Pithecellobium lusorium (Vell.) Benth.'),
 ('Família: ', 'Fabaceae'),
 ('Bioma: ', 'Mata Atlântica'),
 ('Formação Vegetal: ', 'Florestal, Savânica'),
 ('Fitofisionomias: ', 'Floresta Ombrófila Densa, Restinga Arbórea/Arbustiva'),
 ('Presença nos estados: ', 'BA, ES, MG, RJ, SC, SP')]

## Fieldset 2 (Produção de mudas)

In [6]:
labels = fieldsets[1].findAll('td',{ 'class': "pg_infogeral_label"})
values = fieldsets[1].findAll('td',{ 'class': "pg_infogeral_valor"})

fs2=list(zip( [ l.text for l in labels], [v.text for v in values] ))
fs2

[('Período de coleta de sementes: ',
  'Mata Atlântica - Jun - Out (SP); Jul -Dez (PR)'),
 ('Beneficiamento de sementes: ',
  'Os frutos devem ser colhidos diretamente da árvore quando iniciarem abertura espontânea .'),
 ('Porcentagem de germinação: ', '50 a 80% (boa)'),
 ('Substrato da muda: ',
  '\n\n\nCompostos orgânicos (esterco bovino, casca de eucalipto, bagaço de cana) + terra + vermiculita. Mais informações no MANUAL TÉCNICO PARA A RESTAURAÇÃO DE ÁREAS DEGRADADAS NO ESTADO DO RIO DE JANEIRO (Moraes et al., 2012).\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t[Saiba mais]\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\n\n\n'),
 ('Desenvolvimento da muda no viveiro: ', 'Rápido (entre 30 e 100 cm/ano)'),
 ('Número de sementes/kg: ', '1300'),
 ('Armazenamento: ',
  'Sugerimos condições padrão para sementes com tolerância à dessecação: câmara seca (U.R. 20%)'),
 ('Semeadura: ', 'Direta em recipientes individuais'),
 ('Nível de sombreamento da muda no viveiro: ', '')]

## Fieldset 3 (Área de ocorrência)

Skipping...

## Fieldset 4 (Plantio)

In [8]:
labels = fieldsets[3].findAll('td',{ 'class': "pg_infogeral_label"})
values = fieldsets[3].findAll('td',{ 'class': "pg_infogeral_valor"})

fs4=list(zip( [ l.text for l in labels], [v.text for v in values] ))
fs4

[('Tolerância a Sombra: ', 'NÃO (Pioneira típica)'),
 ('Estratégia ecológica de ocupação: ', 'Recobrimento'),
 ('Desenvolvimento da muda no campo: ', 'Lento (até 30 cm/ano)'),
 ('Recomendação de adubação no plantio no campo: ', '')]

## Fieldset 5 (Outras info)

In [7]:
labels = fieldsets[4].findAll('td',{ 'class': "pg_infogeral_label"})
values = fieldsets[4].findAll('td',{ 'class': "pg_infogeral_valor"})

fs5=list(zip( [ l.text for l in labels], [v.text for v in values] ))
fs5

[('Porte da planta: ', 'Árvore (> 3 metros)'),
 ('Período de floração: ', 'Mata Atlântica - Set - jun(PR); Mar - jun (SP)'),
 ('Período de frutificação: ', 'Mata Atlântica - jun (SP), jul (PR).'),
 ('Uso Econômico: ', 'Medicinal'),
 ('Caducifolia: ', 'NÃO'),
 ('Polinização: ', 'Abelhas'),
 ('Dispersão: ', 'Zoocoria (Fauna)')]

# Create the dataset from data from multiple species

In [70]:
species_files = glob.glob('./species_html/*')

In [71]:
def getData(bs):
    data = []
    fieldsets =  [ legend.findParent() for legend in bs.findAll('legend') ]
    
    for fieldset in fieldsets:
        labels = fieldset.findAll('td',{ 'class': "pg_infogeral_label"})
        values = fieldset.findAll('td',{ 'class': "pg_infogeral_valor"})
        data += list(zip( [ l.text.strip() for l in labels], [v.text for v in values] ))
    
    return dict(data)

In [72]:
dataset = []
for spfile in species_files:
    with open(spfile, encoding='utf-8') as f:
        bs = BeautifulSoup( f.read())
    
    dataset += [ getData(bs) ]

In [73]:
def structure_asPandasDf(dataset):
    """
    Structures dataset as a Pandas dataframe
    Input dataset is a list of records, formatted as {column:value} dicts
    """
    # restructure dataset
    new_dataset = {k:[] for r in dataset for k in r.keys() }
    dataset_keys = new_dataset.keys()

    for rec in dataset:
        for k in dataset_keys:
            new_dataset[k].append(rec.get(k,None))
            
    return pd.DataFrame.from_dict(data=new_dataset,orient='columns')

In [74]:
pd.set_option('display.max_columns',30)

In [75]:
df = structure_asPandasDf(dataset)
df.columns = list(map(lambda x: x[:-1], df.columns))
df.replace('', pd.np.NaN, inplace=True)

In [76]:
df

,Espécie,Nome Popular,Sinonímia,Família,Bioma,Formação Vegetal,Fitofisionomias,Presença nos estados,Período de coleta de sementes,Beneficiamento de sementes,Porcentagem de germinação,Substrato da muda,Desenvolvimento da muda no viveiro,Número de sementes/kg,Armazenamento,Semeadura,Nível de sombreamento da muda no viveiro,Tolerância a Sombra,Estratégia ecológica de ocupação,Desenvolvimento da muda no campo,Recomendação de adubação no plantio no campo,Porte da planta,Período de floração,Período de frutificação,Uso Econômico,Caducifolia,Polinização,Dispersão
0,Abarema brachystachya(DC.) Barneby & J.W.Grimes,"Timbuva, Olho-cabra-azul","Inga brachystachya DC., Abarema obovata (Benth...",Fabaceae,Mata Atlântica,"Florestal, Savânica","Floresta Ombrófila Densa, Restinga Arbórea/Arb...","BA, ES, MG, RJ, SC, SP",Mata Atlântica - Jun - Out (SP); Jul -Dez (PR),Os frutos devem ser colhidos diretamente da ár...,50 a 80% (boa),"\n\n\nCompostos orgânicos (esterco bovino, cas...",Rápido (entre 30 e 100 cm/ano),1300,Sugerimos condições padrão para sementes com t...,Direta em recipientes individuais,NaN,NÃO (Pioneira típica),Recobrimento,Lento (até 30 cm/ano),NaN,Árvore (> 3 metros),Mata Atlântica - Set - jun(PR); Mar - jun (SP),"Mata Atlântica - jun (SP), jul (PR).",Medicinal,NÃO,Abelhas,Zoocoria (Fauna)
1,Abarema cochleata(Willd.) Barneby & J.W.Grimes,"Saboeiro, Inga-chumbinho, Tento",Inga cochleata Willd,Fabaceae,Amazônia,"Florestal, Savânica","Campinarana, Savana, Terra Firme","AM, MA, PA, TO",mai-jul,Os frutos devem ser colhidos diretamente da ár...,NaN,\n\n\n3 partes de terra preta+ 1 parte de serr...,Rápido (entre 30 e 100 cm/ano),NaN,NaN,Indireta em sementeira,Pleno Sol,NÃO (Pioneira típica),Diversidade,Rápido (entre 30 e 100 cm/ano),**Amazônia S01,Árvore (> 3 metros),set-out,out-jul,"Madeireiro, Ornamental",NaN,Beija-flores,Zoocoria (Fauna)
2,Abarema floribunda(Spruce ex Benth.) Barneby &...,Tento,Pithecellobium floribundum Spruce ex Benth,Fabaceae,Amazônia,Florestal,Terra Firme,"AC, AM",jan-abr,Os frutos devem ser colhidos diretamente da ár...,NaN,\n\n\n3 partes de terra preta+ 1 parte de serr...,Rápido (entre 30 e 100 cm/ano),NaN,NaN,Indireta em sementeira,Pleno Sol,NÃO (Pioneira típica),Diversidade,Rápido (entre 30 e 100 cm/ano),**Amazônia S01,Árvore (> 3 metros),jul-out,ago-abr,"Madeireiro, Ornamental",NaN,Beija-flores,Zoocoria (Fauna)
3,Abarema jupunba(Willd.) Britton & Killip.,"Angelim-falso, Contas-de-nossa-senhora, Ingara...",Acacia jupunba Willdenow,Fabaceae,Amazônia,"Florestal, Savânica","Restinga, Terra Firme","AC, AM, AP, BA, CE, ES, MA, MG, PA, PB, PE, RJ...",jun-jul,Os frutos devem ser colhidos diretamente da ár...,10 a 19% (baixa),\n\n\n3 partes de terra preta+ 1 parte de serr...,Rápido (entre 30 e 100 cm/ano),1300,NaN,Indireta em sementeira,Pleno Sol,NÃO (Pioneira típica),Diversidade,Rápido (entre 30 e 100 cm/ano),**Amazônia S01,Árvore (> 3 metros),out-jan,jan-jul,"Madeireiro, Ornamental",NÃO,Beija-flores,Zoocoria (Fauna)
4,Abarema piresiiBarneby & J.W.Grimes,Tento,NaN,Fabaceae,Amazônia,"Florestal, Savânica","Campinarana, Terra Firme","AM, MG, PA",jan-jun,Os frutos devem ser colhidos diretamente da ár...,NaN,\n\n\n3 partes de terra preta+ 1 parte de serr...,Rápido (entre 30 e 100 cm/ano),NaN,NaN,Indireta em sementeira,Pleno Sol,NÃO (Pioneira típica),Diversidade,Rápido (entre 30 e 100 cm/ano),**Amazônia S01,Árvore (> 3 metros),mai-set,jan-jun,"Madeireiro, Ornamental",NaN,Beija-flores,Zoocoria (Fauna)
5,Acanthosyris spinescens(Mart. & Eichler) Griseb.,Sombra-de-touro,Osyris spinescens Mart. & Eichler,Santalaceae,Pampa,"Florestal, Savânica","Floresta Estacional, Vegetaçao Savanóide","GO, PR, RS",NaN,Os frutos devem ser colhidos diretamente da ár...,NaN,\n\n\n\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t...,NaN,340,NaN,Indireta em sementeira,NaN,NÃO (Pioneira típica),Diversidade,Lento (até 30 cm/ano),NaN,Árvore (> 3 metros),out-nov,mar-mai,"Alimentício, Latex, Oleaginoso",SIM,insetos,Zoocoria (fauna)
6,Acca sellowiana(O.Berg) Bur

### Write to data file (csv)

In [77]:
pd.DataFrame.to_csv(df, './data_webambiente.csv', sep='\t',index=False)